# Fase1.- Formar BBDD

## Wikipedia
Vamos a probar a construir la descripcion mediante entradas de wikipedia. El punts fuerte de este metodo es que, en caso de encontrar un articulo de la empresa buscada, la informacion será objetiva, facil de obtener y de hacer un resumen. 

Buscaremos los articulos en ingles. Hay articulos que todavia no se han traducido y cabe la posibilidad que nuestra empresa (si es muy  nicho) tenga un articulo en su idioma nativo y no en EN. Adjunto dos links, en el primero hay articulos en cola de ser traducidos y en el segundo es una herramienta que encuentra atributos dentro de un articulo que no son en ingles (este te redirige a la BBDD de 'entities' de wikidata) (simplemente por si se decide explorarlos explorarlos):

- https://not-in-the-other-language.toolforge.org/?lang1=de&proj1=wiki&lang2=en&proj2=wiki&cat=&depth=9&limit=100&starts_with=&start=100&targets=wikidata&doit=1

- https://not-in-the-other-language.toolforge.org/?lang1=de&proj1=wiki&lang2=en&proj2=wiki&cat=&depth=9&limit=100&starts_with=&start=100&targets=wikidata&doit=1

Podemos acceder de distinatas formas a la API de wikipedia (wikimedia). Con la URL o con la API envualta en la libreria *wikipedia*. Hay dos versiones de la AP disponibles, la --v 1.4.0 y la --v 0.6.0. Primero importaremos el POC

In [151]:
import pandas as pd

ruta_csv = '../data/data.csv'
df = pd.read_csv(ruta_csv, sep=',')
print(df.head(10)) 

                           Company name                          Website  \
0                    Oakland University             https://oakland.edu/   
1            TomTom North America, Inc.  https://www.tomtom.com/company/   
2                                    3m              https://www.3m.com/   
3                       DIVCON Controls      https://divconcontrols.com/   
4  San Francisco Elevator Services Inc.              https://sfelev.com/   
5      North Jersey Inspection Services                              NaN   
6                         R Recruitment                              NaN   
7                        Consume 4 Less                              NaN   
8       SK and Company Accountants Ltd.                              NaN   
9                        Themathsurgery       http://themathsurgery.com/   

                                         Description  
0  Oakland University harnesses the transformativ...  
1  If you ever found yourself in a car in the nou... 

El primer problmea que encontramos es que en las busquedas de articlos en la API de wikipedia podremos obtener diversos resultados, de contenido completamente distinto. Sobre los titulos de los articulos podemos leer la info (se ve que se ordenan segun 'experienced editors'):

- https://en.wikipedia.org/wiki/Wikipedia:Article_titles/Criteria_order

Podemos ordenar las paginas con el parametro srsort de 'generator'

In [3]:
import requests

query = 'zara'
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }

data = requests.get(url, params=params).json() 
for i in data['query']['search']:
    print(i['title'], ' - Word count: ', i['wordcount'])

Zara  - Word count:  283
Zara Hatke Zara Bachke  - Word count:  1736
Zara (retailer)  - Word count:  4209
Zara Larsson  - Word count:  5826
Zara Tindall  - Word count:  3068
Zara McDermott  - Word count:  1471
Zara (name)  - Word count:  646
Zara Larsson discography  - Word count:  3268
Zara Home  - Word count:  124
Zara, Turkey  - Word count:  166


En la anterior querry buscamos 'zara' en el buscador de la API y aparecen todos los articulos diferentes. A nosotros nos interesa 'Zara (retailer)' pero a priori no podemos identificar sobre la lista de resultados cual es el que nos interesa, ya que en el .csv POC encontramos universidades, tiendas, ... y demás casuísticas que no podemos controlar.

Un walk around seria filtrar los articulos resultado por algunas categorias hardcodeadas. La api de wikipedia no permite hacer busquedas con una categoria como filtro y además wikimedia ha establecido toda una jerarquia de categorias, por lo que puede que la empresa a buscar esté en la categoria 'universidades de UK' y no en la categoria 'universidades' (pese a estar 'universidades de UK' dentro de la categoria de 'universidades', claro) 

Si que se podrian obtener todos los articulos existentes de la categoria (p.ej) 'Bussiness'. De nuevo, no tenemos total certeza que lo que se busquen sean negocios de esa categoria y tendriamos muchas instancias no utilizadas. 

El meta-parametro 'generator' determina que paginas queremos mostrar. Una idea: podria cruzarse la localizacion (coordenadas) de la empresa y el paremetro de generator: geosearch. Hay que explorarlo

Ejemplo de un full search (texto de todas las entradas antes de la pruimera seccion, lo que se considera el Resumen de wikipedia) de 'Python'

In [60]:
import requests
 
subject = 'Python'
url = 'https://en.wikipedia.org/w/api.php'
params = {
        'origin':'*',
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
        'generator': "search",
        'gsrsearch': subject,
        'gsrwhat':'text',
        'gsrsort': 'relevance'
    }
 
response = requests.get(url, params=params)
data = response.json()

for i in data['query']['pages'].values():
    print('Titulo:',i['title'],':\n', i['extract'])

Titulo: Ball python :
 The ball python (Python regius), also called the royal python, is a python species native to West and Central Africa, where it lives in grasslands, shrublands and open forests. This nonvenomous constrictor is the smallest of the African pythons, growing to a maximum length of 182 cm (72 in). The name "ball python" refers to its tendency to curl into a ball when stressed or frightened.


Titulo: Burmese python :
 The Burmese python (Python bivittatus) is one of the largest species of snakes. It is native to a large area of Southeast Asia and is listed as Vulnerable on the IUCN Red List. Until 2009, it was considered a subspecies of the Indian python, but is now recognized as a distinct species. It is an invasive species in Florida as a result of the pet trade.


Titulo: Colt Python :
 The Colt Python is a double action revolver chambered for the .357 Magnum cartridge. It was first introduced in 1955 by the Colt's Manufacturing Company. Pythons have a reputation fo

### Full text de una pagina

In [49]:
from bs4 import BeautifulSoup
 
subject = 'Python (programming language)'
 
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action': 'parse',
            'page': subject,
            'format': 'json',
            'prop':'text',
            'redirects':''
        }
 
response = requests.get(url, params=params)
data = response.json()
 
raw_html = data['parse']['text']['*']
soup = BeautifulSoup(raw_html,'html.parser')
soup.find_all('p')
text = ''
 
for p in soup.find_all('p'):
    text += p.text
 
print(text[:1000])
print('Text length: ', len(text))


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[31]
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[32][33]
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0.[34] Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.[35]
Python consistently ranks as one of the most popular programming languages.[36][37][38][39]
Python was conceived in the late 1980s[40] by Guido van Rossum at Centrum Wiskunde & 

# Wikidata

In [10]:
import requests
 
subject = 'Python'
url = 'https://en.wikipedia.org/w/api.php'
query = "wikipedia"
params = {
    'action': 'wbsearchentities',
    'format': 'json',
    'language': 'en',
    'search': query
    }
 
response = requests.get(url, params=params)
data = response.json()
print(data)
# page = next(iter(data['query']['pages'].values()))
# print(page['extract'][:73])

{'error': {'code': 'badvalue', 'info': 'Unrecognized value for parameter "action": wbsearchentities.', '*': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}, 'servedby': 'mw-api-ext.eqiad.main-675759d54b-hsss7'}


## Extra

A priorio solo querríamos obtener el summary de una pagina o su texto al completo. Con las siguientes funciones podemos obtener texto por las secciones definidas de una pagina 

In [12]:
import wikipediaapi
def get_sections(
     section: wikipediaapi.WikipediaPageSection,
     parent_title: list[str],
     sections_to_ignore: set[str]
     ) -> list[tuple[list[str],str]]:
    
     """Gather sections and subsections data."""
    
     sect_title = section.title 
     title = parent_title
     title.extend([sect_title])
     results = []
     
     if sect_title not in sections_to_ignore:
       sect_text = section.text 
       string_section = ( title , sect_text)
       results.extend([string_section])
  
       if len(section.sections)>1:
         for subsection in section.sections:
           get_sections(subsection, title, sections_to_ignore) 

     return results

def get_pages (
     page: wikipediaapi.WikipediaPage,
     sections_to_ignore: set[str]
     )-> list[tuple[list[str],str]]:

     """Gather the page information: title and summary, and then go deep in sections information."""
   
     parent_title = page.title
     print(f"parent_title = {parent_title} ")

     summary = page.summary
     string_parent = ([parent_title], summary)
     results=[string_parent]
     
     if len(page.sections) > 0:
       for section in page.sections:
         results.extend(get_sections(section,[parent_title], sections_to_ignore))
    
     return results

In [18]:
SECTIONS_TO_IGNORE = [
 "See also", "References", "External links",
 "Further reading", "Footnotes", "Bibliography",
 "Sources", "Citations", "Literature",
 "Photo gallery", "Photos", "Gallery",
 "Works cited", "Notes", "Notes and references",
 "References and sources", "References and notes",
]
wiki = wikipediaapi.Wikipedia('ExtractBussinesInfo (ramoncormo8@gmial.com)', language='en')
CATEGORY_TITLE = "Category:2022 FIFA World Cup"

# Get the category object for "2022 FIFA World Cup"
cat = wiki.page(CATEGORY_TITLE)

# Print the name of the category
print("Name: ", cat.categorymembers)
# Get the articles included in this category
# # if it's an article: wikipediaapi.Namespace.MAIN = 0
# # if it's a subcategory: wikipediaapi.Namespace.CATEGORY = 14
articles = [w for w in cat.categorymembers.values() if w.ns == wikipediaapi.Namespace.MAIN]

wiki_corpus=[]
for page in articles:
    wiki_corpus.extend(get_pages(page, SECTIONS_TO_IGNORE))

wiki_corpus[:5]

Name:  {'2022 FIFA World Cup': 2022 FIFA World Cup (id: ??, ns: 0), '2015 FIFA corruption case': 2015 FIFA corruption case (id: ??, ns: 0), '2022 FIFA World Cup opening ceremony': 2022 FIFA World Cup opening ceremony (id: ??, ns: 0), 'Adidas Al Rihla': Adidas Al Rihla (id: ??, ns: 0), 'Allegations of corruption and bribery related to the 2022 FIFA World Cup': Allegations of corruption and bribery related to the 2022 FIFA World Cup (id: ??, ns: 0), 'Battle of Lusail': Battle of Lusail (id: ??, ns: 0), '2018 and 2022 FIFA World Cup bids': 2018 and 2022 FIFA World Cup bids (id: ??, ns: 0), '2022 FIFA World Cup broadcasting rights': 2022 FIFA World Cup broadcasting rights (id: ??, ns: 0), "Criticisms of Qatar's suitability to host the 2022 FIFA World Cup": Criticisms of Qatar's suitability to host the 2022 FIFA World Cup (id: ??, ns: 0), 'List of 2022 FIFA World Cup controversies': List of 2022 FIFA World Cup controversies (id: ??, ns: 0), 'Dreamers (Jungkook song)': Dreamers (Jungkook son

[(['2022 FIFA World Cup'],
  "The 2022 FIFA World Cup was the 22nd FIFA World Cup, the world championship for national football teams organized by FIFA. It took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan.This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot climate, the event was held during November and December instead of in the traditional months of May, June, or July. It was held over a reduced time frame of 29 days with 64 matches played in eight venues across five cities. Qatar entered the event—their first World Cup—automatically as the host's national team, alongside 31 teams determined by the qualification process.\nArgentina were crowned the 

## Funcion para parsear respuestas

He tomado estas decisiones (a priori): Usar el endpoint URL y la libreria wikipedia --v 1.4.0 para el suggest; Si hay mas de una respeusta coger la info de la primera respuesta con un '**WARNING - need to check veracity**' al principio (hay que mirar mas a fondo el parametro gsrsort o alguno de sorting); Si no hay articulos relacionados al input, aplica la funcion suggest de la libreria wikipedia-api y vuelve a probar; Si no hay respuesta devuelve: 'No content'

Caveats: hay que definir dos veces los parametros, ya que sino no funciona (una cosa es buscar los titulos y otra es buscar el abstract de un titulo en concreto)

In [86]:
import wikipedia
print(wikipedia.suggest("qwerqwecq"))

None


In [67]:
subject = 'Python'
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }
 
response = requests.get(url, params=params)
data = response.json()
 
page = next(iter(data['query']['pages'].values()))
print(page['extract'])
print(dict(data['query']['pages']).keys())

Python may refer to:


dict_keys(['46332325'])


In [98]:
import requests

query = 'Zara (retailer)'
url = 'https://en.wikipedia.org/w/api.php'
params = {
            'action':'query',
            'format':'json',
            'generator': "search",
            'utf8':1,
            'gsrsearch': subject,
            'prop': 'extracts',
            'exintro': True,
        }

data = requests.get(url, params=params).json() 
print(data)
# for i in data['query']['search']:
#     print(i['title'])

{'batchcomplete': '', 'continue': {'gsroffset': 10, 'continue': 'gsroffset||'}, 'warnings': {'extracts': {'*': 'HTML may be malformed and/or unbalanced and may omit inline images. Use at your own risk. Known problems are listed at https://www.mediawiki.org/wiki/Special:MyLanguage/Extension:TextExtracts#Caveats.'}}, 'query': {'pages': {'271890': {'pageid': 271890, 'ns': 0, 'title': 'Ball python', 'index': 4, 'extract': '<p class="mw-empty-elt">\n\n\n</p>\n<p>The <b>ball python</b> (<i><b>Python regius</b></i>), also called the <b>royal python</b>, is a python species native to West and Central Africa, where it lives in grasslands, shrublands and open forests. This nonvenomous constrictor is the smallest of the African pythons, growing to a maximum length of 182\xa0cm (72\xa0in). The name "ball python" refers to its tendency to curl into a ball when stressed or frightened.</p>\n\n\n'}, '819149': {'pageid': 819149, 'ns': 0, 'title': 'Burmese python', 'index': 6, 'extract': '<p class="mw-e

In [150]:
import requests

subject = 'Python'
url = 'https://en.wikipedia.org/w/api.php'
params = {
        'origin':'*',
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
        'exlimit': 1,
        "redirects": 1,
        'generator': "search",
        'gsrsearch': subject,
    }
 
response = requests.get(url, params=params)
data = response.json()
print(list(data['query']['pages'].values())[0]['extract'])
for i in data['query']['pages'].values():
    print(i)
    #print('Titulo:',i['title'],':\n', i['extract'])

The ball python (Python regius), also called the royal python, is a python species native to West and Central Africa, where it lives in grasslands, shrublands and open forests. This nonvenomous constrictor is the smallest of the African pythons, growing to a maximum length of 182 cm (72 in). The name "ball python" refers to its tendency to curl into a ball when stressed or frightened.


{'pageid': 271890, 'ns': 0, 'title': 'Ball python', 'index': 5, 'extract': 'The ball python (Python regius), also called the royal python, is a python species native to West and Central Africa, where it lives in grasslands, shrublands and open forests. This nonvenomous constrictor is the smallest of the African pythons, growing to a maximum length of 182 cm (72 in). The name "ball python" refers to its tendency to curl into a ball when stressed or frightened.\n\n'}
{'pageid': 819149, 'ns': 0, 'title': 'Burmese python', 'index': 6}
{'pageid': 639888, 'ns': 0, 'title': 'Colt Python', 'index': 10}
{'pageid

In [143]:
subject = 'Debian version history'
url = 'https://en.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
 
response = requests.get(url, params=params)
data = response.json()
 
page = next(iter(data['query']['pages'].values()))
print(page['extract'])

Debian releases do not follow a fixed schedule. Recent releases have been made roughly biennially by the Debian Project. The most recent version of Debian is Debian version 12, codename "Bookworm". The next up and coming release of Debian is Debian 13, codename "Trixie".Debian always has at least three active branches at any time: "stable", "testing" and "unstable". The stable branch is considered the primary release and what most people refer to when talking about Debian. The testing branch contains packages that have been imported from unstable. Testing has significantly more up-to-date packages than stable and is frozen some time before a release to become the next version of Debian. The unstable release (also known as Sid) is the branch where active development takes place. It is the most volatile version of Debian.
When the Debian stable branch is replaced with a newer release, the current stable becomes an "oldstable" release. When the Debian stable branch is replaced again, the 

In [153]:
import wikipedia 
import time
def wikipedia_info(subject: str) -> str:
    time.sleep(50)
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
            'origin':'*',
            'action': 'query',
            'format': 'json',
            'prop': 'extracts',
            'exintro': True,
            'explaintext': True,
            "exlimit": 1, 
            'generator': "search",
            'gsrsearch': subject,
            'gsrwhat':'text',
            'gsrsort': 'relevance'
        }
    data = requests.get(url, params=params).json() 

    if "query" in data:
        if len(data['query']['pages'])>1:
            warnings = '**WARNING - need to check veracity**'
            summary = warnings + list(data['query']['pages'].values())[0]['extract']
            return summary
        else:
            params = {
                    'action': 'query',
                    'format': 'json',
                    'titles': subject,
                    'prop': 'extracts',
                    'exintro': True,
                    'explaintext': True,
                }
            
            data = requests.get(url, params=params).json()            
            page = next(iter(data['query']['pages'].values()))
            return page['extract']
    else: 
        if wikipedia.suggest(subject) is not None:
            subject_suggested = wikipedia.suggest(subject)
            return wikipedia_info(subject_suggested)
        else: 
            return 'No match'

In [154]:
result_dict = {}
for company_name in df['Company name']:
    result = wikipedia_info(company_name)
    result_dict[company_name] = result

In [ ]:
result_dict

In [ ]:
import time
import requests
import logging
import json
from typing import Dict

logger = logging.getLogger(__name__)
logger.level = logging.INFO


URL = 'https://en.wikipedia.org/w/api.php'

search_list_params = {
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
        "exlimit": 1, 
        'generator': "search",
        'gsrsearch': 'subject',
}
search_item_params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
}

def wikipedia_info(subject: str):
    time.sleep(10)
    data = request_with_cooloff(url=URL, params=search_list_params)

    if "query" in data:
        if len(data['query']['pages'])>1:
            warnings = '**WARNING - need to check veracity**'
            summary = warnings + '\n' + list(data['query']['pages'].values())[0]['extract']
            return summary
        else:
            warnings = '**NO WARNING - just one item left**'
            data = request_with_cooloff(url=URL, params=search_item_params)         
            page = next(iter(data['query']['pages'].values()))
            return page['extract']
    else: 
        if wikipedia.suggest(subject) is not None:
            subject_suggested = wikipedia.suggest(subject)
            return wikipedia_info(subject_suggested)
        else: 
            return 'No subject found in Wiki API'

def _request_with_cooloff(
    url: str, params: Dict[str, any], num_attempts: int, payload: Dict[str, any] = None
):
    """
    Call the url using requests. If the endpoint returns an error wait a cooloff
    period and try again, doubling the period each attempt up to a max num_attempts.
    """
    cooloff = 1
    for call_count in range(num_attempts):
        try:
            response = requests.get(url,params=params)
            response.raise_for_status()

        # If we're overloading the endpoint it may refuse a connection
        except requests.exceptions.ConnectionError as e:
            logger.info("API refused the connection")
            logger.warning(e)
            if call_count != (num_attempts - 1):
                time.sleep(cooloff)
                cooloff *= 2
                call_count += 1
                continue
            else:
                raise

        # Catch non 200 return codes on the HTTP header
        except requests.exceptions.HTTPError as e:
            logger.warning(e)
            if response.status_code == 404:
                raise

            logger.info(f"API return code {response.status_code} cooloff at {cooloff}")
            if call_count != (num_attempts - 1):
                time.sleep(cooloff)
                cooloff *= 2
                call_count += 1
                continue
            else:
                raise

        # We got through the loop without error so we've received a valid response
        return response

def request_with_cooloff(
    url: str,
    params: Dict[str, any],
    num_attempts: int = 10,
):
    return json.loads(
        _request_with_cooloff(
            url,
            params,
            num_attempts
        ).content.decode("utf-8")
    )

def main():
    result_dict = {}
    for company_name in df['Company name']:
        result = wikipedia_info(company_name)
        result_dict[company_name] = result
    return result_dict